In [ ]:
### ENTROPY

density_list = np.linspace(0.3, 0.9, 100)
L = 100
T = 50
n_repetitions = 100
v_max = 5
trials = 5
shannon_entropy = {}

import itertools

for p in density_list:
    print(p)
    for n in range(trials):
        lifespan_counter, jam_counter, all_evolutions = run_model_stochastic(p, L, T, n_repetitions, v_max, return_evolutions=True)
        merged = list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(all_evolutions))))
        second_items = [tuple_item[1] for tuple_item in merged]
        speeds = Counter(second_items)
        
        probabilities = [count / len(second_items) for count in speeds.values()]

        # shannon_entropy[p] += -sum(p * np.log2(p) for p in probabilities)

        if p not in shannon_entropy:
                    shannon_entropy[p] = [-sum(p * np.log2(p) for p in probabilities)]
        else:
            shannon_entropy[p] += [-sum(p * np.log2(p) for p in probabilities)]

In [ ]:
def double_fit(x, y):
    highest_b2 = 0

    for index in range(int(len(x)*0.2), int(len(x)*0.8)):
        popt1, pcov1 = scipy.optimize.curve_fit(func, x[0:index], y[0:index], p0=[1, 0.5], maxfev=5000)
        a1, b1 = popt1

        popt2, pcov2 = scipy.optimize.curve_fit(func, x[index:], y[index:], p0=[1, 0.5], maxfev=5000)
        a2, b2 = popt2

        if b2 > highest_b2:
            # Find steepest line
            highest_b2 = b2

            best_a1, best_b1, best_a2, best_b2 = a1, b1, a2, b2

            optimal_index = index

    return best_a1, best_b1, best_a2, best_b2, optimal_index

def find_intersection(x_data, a1, b1, a2, b2):
    # Finds the first intersection which will automatically be the point where y1 > y2
    x = np.linspace(1, np.max(x_data), np.max(x_data))
    y1 = func(x, a1, b1)
    y2 = func(x, a2, b2)
    for index in range(len(x)):
        if y1[index] >= y2[index]:
            return x[index]
        
    return None

def plot_fit(x, y, a1, b1, a2, b2):
    plt.clf()
    plt.loglog(x, y, 'o', markersize=2)
    plt.xlim(0, 1.1*np.max(x))
    plt.ylim(0, 1.1*np.max(y))
    plt.loglog(x, func(x, a1, b1))
    plt.loglog(x, func(x, a2, b2))
    plt.show()

def find_critical_size(density_list, n_simulations, L, T, n_repetitions):
    critical_size_dict = {}
    critical_size = None
    for p in density_list:
        for n in range(n_simulations): 
            print(p, n, critical_size)
            lifespan_counter, final_jams_counted = run_model_stochastic(p, L, T, n_repetitions, v_max=5)
            lists = sorted(final_jams_counted.items()) # sorted by key, return a list of tuples

            x_data, y_data = zip(*lists)
            a_optimal1, b_optimal1, a_optimal2, b_optimal2, optimal_index = double_fit(x_data, y_data)

            critical_size = find_intersection(x_data, a_optimal1, b_optimal1, a_optimal2, b_optimal2)

            if critical_size == None:
                # If lines do not intersect skip to the next density
                if p in critical_size_dict:
                    del critical_size_dict[p]        
                break

            else:
                plot_fit(x_data, y_data, a_optimal1, b_optimal1, a_optimal2, b_optimal2)
                if p not in critical_size_dict:
                    critical_size_dict[p] = [critical_size]
                else:
                    critical_size_dict[p] += [critical_size]

    return critical_size_dict

def double_fit_saver(x, y):
    highest_b2 = 0

    for index in range(int(len(x)*0.2), int(len(x)*0.8)):
        popt1, pcov1 = scipy.optimize.curve_fit(func, x[0:index], y[0:index], p0=[1, 0.5], maxfev=5000)
        a1, b1 = popt1

        popt2, pcov2 = scipy.optimize.curve_fit(func, x[index:], y[index:], p0=[1, 0.5], maxfev=5000)
        a2, b2 = popt2

        if b2 > highest_b2:
            # Find steepest line
            highest_b2 = b2

            best_a1, best_b1, best_a2, best_b2 = a1, b1, a2, b2

            optimal_index = index

        plot_fit_saver(x, y, a1, b1, a2, b2, index)   
        
    return best_a1, best_b1, best_a2, best_b2, optimal_index